## Setup

In [1]:
!nvidia-smi

Thu Sep 12 07:06:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Install libraries

In [2]:
%pip install --upgrade --quiet langchain langchain_community pdfplumber chromadb tqdm streamlit ollama pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 95.8 MB/s eta 0:00:00


In [3]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Installing ollama to /usr/local
100 13320    0 13320    0     0  45155      0 --:--:-- --:--:-- --:--:-- 45152
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
!ollama serve > rocama.log 2>&1 &
!ollama pull llama3 & ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕▏    0 B/274 MB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕▏    0 B/274 MB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕▏  73 KB/274 MB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏ 2.5 MB/4

In [5]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED               
llama3:latest          	365c0bd3c000	4.7 GB	Less than a second ago	
nomic-embed-text:latest	0a109f422b47	274 MB	36 seconds ago        	


### Load libraries

In [6]:
import os
import datetime
from tqdm import tqdm
import tempfile
import streamlit as st
from IPython.display import Markdown

import ollama
from langchain.chains import RetrievalQA
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

### Constants

In [7]:
# Define the directory where your PDFs are stored
pdf_directory = "/content/sample_data/pdfs/"
save_dir = pdf_directory

vector_database_path = "/content/sample_data/vector_database"

for path in [pdf_directory, vector_database_path]:
  if not os.path.exists(path):
    os.makedirs(path, exist_ok=True)

## Workflow

### 1. Extract Texts from PDFs

In [26]:
# Get a list of all PDF files in the directory
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

pdf_pages = []
# Iterate through each PDF file and load it
for pdf_file in pdf_files:
    file_path = os.path.join(pdf_directory, pdf_file)
    print(f"Loading file: {pdf_file}")

    # Load the PDF and split it into pages
    loader = PDFPlumberLoader(file_path=file_path)
    pages = loader.load()
    pdf_pages.extend(pages)

Loading file: Advances in Emerging Memory Technologies - From Data Storage to Artificial Intelligence.pdf


### 2. Split Text

In [27]:
# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False
)

text_chunks = []
# Split the pages into chunks
for page in pdf_pages:
  chunks = text_splitter.split_text(page.page_content)
  text_chunks.extend(chunks)

In [28]:
text_chunks

['applied\nsciences\nReview\nAdvances in Emerging Memory Technologies: From Data\nStorage to Artificial Intelligence\nGabrielMolas*andEtienneNowak\nCEA,LETI,UniversitéGrenobleAlpes,38000Grenoble,France;etienne.nowak@cea.fr\n* Correspondence:gabriel.molas@cea.fr\nAbstract: Thispaperpresentsanoverviewofemergingmemorytechnologies. Itbeginswiththe\npresentationofstand-aloneandembeddedmemorytechnologyevolution,sincetheappearance',
 'of Flash memory in the 1980s. Then, the progress of emerging memory technologies (based on\nfilamentary,phasechange,magnetic,andferroelectricmechanisms)ispresentedwithareviewofthe\nmajordemonstrationsintheliterature.Thepotentialofthesetechnologiesforstorageapplications\naddressingvariousmarketsandproductsisdiscussed.Finally,wediscusshowtheriseofartificial\nintelligenceandbio-inspiredcircuitsoffersanopportunityforemergingmemorytechnologyand',
 'shiftstheapplicationfrompuredatastoragetostorageandcomputingtasks,andalsoenlargesthe\nrangeofrequiredspecificationsatthe

In [29]:
# Metadata management
def add_metadata(chunks, doc_title):
  metadata_chunks = []
  for chunk in chunks:
    metadata = {
        "title": doc_title,
        "author": "Gabriel Molas, Etienne Nowak",
        "date": str(datetime.date.today())
    }
    metadata_chunks.append({"text": chunk, "metadata": metadata})
  return metadata_chunks

# Add metadata to chunks
chunks_with_metadata = add_metadata(text_chunks, "Advances in Emerging Memory Technologies - From Data Storage to Artificial Intelligence")

In [30]:
chunks_with_metadata

[{'text': 'applied\nsciences\nReview\nAdvances in Emerging Memory Technologies: From Data\nStorage to Artificial Intelligence\nGabrielMolas*andEtienneNowak\nCEA,LETI,UniversitéGrenobleAlpes,38000Grenoble,France;etienne.nowak@cea.fr\n* Correspondence:gabriel.molas@cea.fr\nAbstract: Thispaperpresentsanoverviewofemergingmemorytechnologies. Itbeginswiththe\npresentationofstand-aloneandembeddedmemorytechnologyevolution,sincetheappearance',
  'metadata': {'title': 'Advances in Emerging Memory Technologies - From Data Storage to Artificial Intelligence',
   'author': 'Gabriel Molas, Etienne Nowak',
   'date': '2024-09-12'}},
 {'text': 'of Flash memory in the 1980s. Then, the progress of emerging memory technologies (based on\nfilamentary,phasechange,magnetic,andferroelectricmechanisms)ispresentedwithareviewofthe\nmajordemonstrationsintheliterature.Thepotentialofthesetechnologiesforstorageapplications\naddressingvariousmarketsandproductsisdiscussed.Finally,wediscusshowtheriseofartificial\ninte

### 3. Create embeddings from text chunks

In [31]:
# Generate embeddings for text chunks
def generate_embeddings(text_chunks, model_name='nomic-embed-text'):
  embeddings = []
  for chunk in tqdm(text_chunks, desc="Embedding text chunks"):
    embedding = ollama.embeddings(model=model_name, prompt=chunk)
    embeddings.append(embedding)
  return embeddings

In [32]:
texts = [chunk["text"] for chunk in chunks_with_metadata]
embeddings = generate_embeddings(texts)

Embedding text chunks: 100%|██████████| 252/252 [00:25<00:00,  9.70it/s]


### 4. Store and use embeddings in ChromaDB

In [33]:
# Wrap texts with their respective metadata into Document objects
documents = [Document(page_content=chunk['text'], metadata=chunk['metadata']) for chunk in chunks_with_metadata]

vector_store = Chroma.from_documents(documents=documents,
                                    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=False),
                                    persist_directory=vector_database_path,
                                    collection_name="local-rag"
)

vector_store.persist()

### 5. Query Processing Multi-Query Retriever

In [34]:
# LLM from Ollama
local_model = "llama3"
llm = ChatOllama(model=local_model)

In [35]:
QUERY_PROMPT = PromptTemplate(
  input_variables=["question"],
  template="""You are an AI language model assistant. Your task is to generate five
  different versions of the given user question to retrieve relevant documents from
  a vector database. By generating multiple perspectives on the user question, your
  goal is to help the user overcome some of the limitations of the distance-based
  similarity search. Provide these alternative questions separated by newlines.
  Original question: {question}""",
)

In [36]:
retriever = MultiQueryRetriever.from_llm(
  vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 4, "fetch_k": 10}),
  ChatOllama(model=local_model),
  prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [37]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [50]:
questions = '''What is NAND flash memory?'''

rows = []
for document_score in vector_store.similarity_search_with_score(questions):
  rows.append((document_score[0].page_content, document_score[1]))

import pandas as pd
df = pd.DataFrame(rows, columns=['Text', 'Score'])
df

,Text,Score
0,and all manner of flash card products. For sta...,363.202332
1,"DRAM and Flash, often referred to as storage c...",386.425232
2,"The memory is faster than NAND, has higher end...",407.259949
3,in the order of 30–50 ns and the one of flash-...,413.397888


In [51]:
import langchain
langchain.debug = True

display(Markdown(chain.invoke({"question": questions})))

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What is NAND flash memory?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "question": "What is NAND flash memory?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "question": "What is NAND flash memory?"
}[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > retriever:MultiQueryRetriever > chain:RunnableSequence] Entering Chain run with input:
{
  "question": {
    "question": "What is NAND flash memory?"
  }
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "question": {
    "question": "What is NAND flash memory?"
  }
}
[chain/end] [chain:RunnableSequen

Based on the provided context, NAND flash memory is mentioned in a document titled "Advances in Emerging Memory Technologies - From Data Storage to Artificial Intelligence". According to the text:

"...filling the latency gap between DRAM and NAND..."

This suggests that NAND flash memory is a type of storage device with slower access times compared to DRAM (Dynamic Random Access Memory) but faster than PCRAM (Phase Change RAM).

## Streamlit

### Build Streamlit app

In [52]:
%%writefile app.py
import os
import datetime
from tqdm import tqdm
import tempfile
import streamlit as st
from IPython.display import Markdown

import ollama
from langchain.chains import RetrievalQA
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create a Streamlit app example
st.set_page_config(page_title="RAG with LLaMa", layout="wide")
with st.sidebar:
  st.write("**RAG with LLaMa**")

# Metadata management
def add_metadata(chunks, doc_title):
  metadata_chunks = []
  for chunk in chunks:
    metadata = {
        "title": doc_title,
        "author": "company",
        "date": str(datetime.date.today())
    }
    metadata_chunks.append({"text": chunk, "metadata": metadata})
  return metadata_chunks

@st.cache_resource(ttl="1h")
def configure_retriever(uploaded_files):
  # read documents
  docs = []
  temp_dir = tempfile.TemporaryDirectory()
  for file in uploaded_files:
    temp_filepath = os.path.join(temp_dir.name, file.name)
    with open(temp_filepath, "wb") as f:
      f.write(file.getvalue())
    loader = PDFPlumberLoader(file_path=temp_filepath)
    docs.extend(loader.load())

  # split documents
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
  splits = text_splitter.split_documents(docs)

  # create embeddings and store in vectordb
  embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=False)
  vector_db = Chroma.from_documents(splits, embeddings)

  # define retriever
  retriever = vector_db.as_retriever(
    search_type="mmr", search_kwargs={"k": 4, "fetch_k": 10}
  )

  return retriever

uploaded_files = st.sidebar.file_uploader(
  label="Upload PDF files", type=["pdf"], accept_multiple_files=True
)

if not uploaded_files:
  st.info("Please upload PDF documents to continue.")
  st.stop()

# LLM from Ollama
local_model = "llama3"
llm = ChatOllama(model=local_model)

# Prompt template
QUERY_PROMPT = PromptTemplate(
  input_variables=["question"],
  template="""You are an AI language model assistant. Your task is to generate five
  different versions of the given user question to retrieve relevant documents from
  a vector database. By generating multiple perspectives on the user question, your
  goal is to help the user overcome some of the limitations of the distance-based
  similarity search. Provide these alternative questions separated by newlines.
  Original question: {question}""",
)
# Retriever
retriever = MultiQueryRetriever.from_llm(configure_retriever(uploaded_files), llm, prompt=QUERY_PROMPT)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
  {"context": retriever, "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)

msgs = StreamlitChatMessageHistory()

if len(msgs.messages) == 0 or st.sidebar.button("New Chat"):
  msgs.clear()
  msgs.add_ai_message("How can I help you?")

avatars = {"human": "user", "ai": "assistant"}
for msg in msgs.messages:
  st.chat_message(avatars[msg.type]).write(msg.content)

if user_query := st.chat_input(placeholder="Ask me anything!"):
  msgs.add_user_message(user_query)
  st.chat_message("user").write(user_query)

  with st.chat_message("assistant"):
    answer = chain.invoke({"question": user_query})
    msgs.add_ai_message(answer)
    st.write(answer)


about = st.sidebar.expander("About")
about.write("You can easily chat with a PDF using this AI chatbot. ")

Overwriting app.py


### Run Streamlit app with ngrok

In [53]:
from pyngrok import ngrok
from google.colab import userdata

# Set authentication token if you haven't already done so
ngrok.set_auth_token(userdata.get('NGROK_AUTH_KEY'))

# Start Streamlit server on a specific port
!nohup streamlit run app.py --server.port 8501 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='8501', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://0a1e-34-125-136-140.ngrok-free.app
